In [ ]:
from pyspark.sql import functions as F

account =  # your storage account name
container =  # your container name
silver = f"abfss://{container}@{account}.dfs.core.windows.net/silver"
gold   = f"abfss://{container}@{account}.dfs.core.windows.net/gold"

op = spark.read.parquet(f"{silver}/fact_order_products")

# product popularity + reorder ratio
prod_feats = (op.groupBy("product_id")
                .agg(
                    F.count("*").alias("prod_times_ordered"),
                    F.avg(F.col("reordered").cast("double")).alias("prod_reorder_ratio")
                ))

(prod_feats.repartition(1)
    .write.mode("overwrite")
    .parquet(f"{gold}/product_features"))

print("gold.product_features written")